In [176]:
import pandas as pd
import numpy as np
import ast

In [177]:
df=pd.read_csv('./data/messy_data.csv')
df.head()

,id,age,height,weight,income,purchases,gender
0,1,25.0,175.0,68,35000.0,"23,45,12",male
1,2,NaN,182.0,80,42000.0,"34,,19",female
2,3,150.0,165.0,180,28000.0,NaN,unknown
3,4,34.0,168.0,70,NaN,",55,67",male
4,5,29.0,190.0,165,31000.0,"12,19,23",female


import numpy as np	Enables missing values handling
def convert_purchases(x)	Function for one row
pd.isna(x)	Detects NaN
x.strip() == ""	Detects empty strings
x.split(',')	Separates values
isdigit()	Keeps only valid numbers
int(i)	Converts to integer
.apply()	Applies function to each row

<h2>converting  purchases : string to list of data </h2>

In [178]:
def convert_purchases(x):
    if pd.isna(x) or x.strip() == "":
        return np.nan
    return [int(i) for i in x.split(',') if i.strip().isdigit()]

df['purchases'] = df['purchases'].apply(convert_purchases)
df.head()


,id,age,height,weight,income,purchases,gender
0,1,25.0,175.0,68,35000.0,"[23, 45, 12]",male
1,2,NaN,182.0,80,42000.0,"[34, 19]",female
2,3,150.0,165.0,180,28000.0,NaN,unknown
3,4,34.0,168.0,70,NaN,"[55, 67]",male
4,5,29.0,190.0,165,31000.0,"[12, 19, 23]",female


<h2>handling outliers</h2>

In [179]:
max_age=100


In [180]:
df['age'] = df['age'].apply(lambda x : min(x,max_age))
df.head()

,id,age,height,weight,income,purchases,gender
0,1,25.0,175.0,68,35000.0,"[23, 45, 12]",male
1,2,NaN,182.0,80,42000.0,"[34, 19]",female
2,3,100.0,165.0,180,28000.0,NaN,unknown
3,4,34.0,168.0,70,NaN,"[55, 67]",male
4,5,29.0,190.0,165,31000.0,"[12, 19, 23]",female


<h2>handling missing data </h2>

<p> age : group based by gender , median </br>
    income: can group by age and avg of that group </br>
    purchases : can check the income and make it similar to the close income person </p>

In [181]:
#age

df['age'] = df['age'].fillna(df.groupby('gender')['age'].transform('mean'))

df['age'].head()

0     25.0
1     29.0
2    100.0
3     34.0
4     29.0
Name: age, dtype: float64

In [182]:
#income
#using bins for grouping ages in order to determine the income 

age_bins = [0, 24, 50, 75, 100]
age_labels = ['0-24', '25-50', '31-75', '76-100']

df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels)


In [183]:
df['income'] = df['income'].fillna(
    df.groupby('age_group', observed=True)['income'].transform('median'))

df['income'].head()


0    35000.0
1    42000.0
2    28000.0
3    37000.0
4    31000.0
Name: income, dtype: float64

In [184]:
df.head()

,id,age,height,weight,income,purchases,gender,age_group
0,1,25.0,175.0,68,35000.0,"[23, 45, 12]",male,25-50
1,2,29.0,182.0,80,42000.0,"[34, 19]",female,25-50
2,3,100.0,165.0,180,28000.0,NaN,unknown,76-100
3,4,34.0,168.0,70,37000.0,"[55, 67]",male,25-50
4,5,29.0,190.0,165,31000.0,"[12, 19, 23]",female,25-50


In [185]:
#purchases 





In [186]:
df['gender']=df['gender'].apply(lambda g: 1 if g=='male' else 0)

df['gender'].head()


0    1
1    0
2    0
3    1
4    0
Name: gender, dtype: int64

In [187]:
df.head()


,id,age,height,weight,income,purchases,gender,age_group
0,1,25.0,175.0,68,35000.0,"[23, 45, 12]",1,25-50
1,2,29.0,182.0,80,42000.0,"[34, 19]",0,25-50
2,3,100.0,165.0,180,28000.0,NaN,0,76-100
3,4,34.0,168.0,70,37000.0,"[55, 67]",1,25-50
4,5,29.0,190.0,165,31000.0,"[12, 19, 23]",0,25-50


<h2>Normalizing</h2>

In [ ]:
#scaling age,height,weight,income
max_age=df['age'].max()
max_height=df['height'].max()
max_weight=df['weight'].max()
max_income=df['income'].max()


In [189]:
max_age = max_age if max_age !=0 else 1
max_height = max_height if max_height !=0 else 1
max_weight = max_weight if max_weight !=0 else 1
max_income = max_income if max_income !=0 else 1

In [190]:
df['age'] = df['age']/max_age
df['height'] = df['height']/max_height
df['weight'] = df['weight']/max_weight
df['income'] = df['income']/max_income

In [191]:
df.head()

,id,age,height,weight,income,purchases,gender,age_group
0,1,0.25,0.921053,0.357895,0.833333,"[23, 45, 12]",1,25-50
1,2,0.29,0.957895,0.421053,1.000000,"[34, 19]",0,25-50
2,3,1.00,0.868421,0.947368,0.666667,NaN,0,76-100
3,4,0.34,0.884211,0.368421,0.880952,"[55, 67]",1,25-50
4,5,0.29,1.000000,0.868421,0.738095,"[12, 19, 23]",0,25-50


In [192]:
df_new ='./data/dirtydata_cleaned.json'
df.to_json(df_new,orient='records',indent=4)  #saves cleaned data to new json file

print(f'the cleaned data has been saved to {df_new}')

the cleaned data has been saved to ./data/dirtydata_cleaned.json
